A faire : 
- remplir le csv des débris avec leur param orbitaux
- trouver les fonctions pour calculer les delta V
- extraire les donneés du csv et calculer les delta V
- réfléchir à l'optimisation et choisir un algo de regroupement approrié  

In [15]:
import numpy as np
from numpy import linalg as LA

import recoveringDebrisData2 as RDB



In [18]:
debris_data = RDB.convertTLEtoDF(RDB.recoveringDebrisData())

debris_data

,a (km),e,i (rad),RAAN (rad),Argument of periapsis (rad),Mean anomaly (rad),Date (year),Date (fraction of year),Mass (kg)
27001,7345.844298,0.001741,1.447684,2.551294,1.280925,0.657747,2021,346.94114,2500
27601,7355.441145,0.001432,1.44837,2.511665,0.392128,1.502516,2021,346.927092,3000
15334,7363.723312,0.002163,1.447576,1.961436,2.479577,5.79752,2021,346.9014,2440
10732,7351.052891,0.002239,1.448389,6.274588,2.295827,0.717789,2021,346.79536,1435
24279,7358.544255,0.001904,1.447436,1.982554,6.164742,2.074552,2021,346.859171,2700
21090,7352.573825,0.002995,1.447595,1.829704,3.349081,5.95727,2021,347.131392,1435
15772,7362.148355,0.004121,1.447958,0.724936,1.604263,4.68918,2021,346.894133,2440
10693,7221.525963,0.007149,1.418145,3.384923,6.212838,1.933099,2021,346.94107,1435
27387,7353.092565,0.002797,1.447625,1.890249,5.996119,3.32006,2021,347.140737,2575
7594,7361.958872,0.002109,1.447349,4.027562,3.397521,4.916271,2021,347.238468,1435


In [ ]:
len(debris_data)

19

In [37]:
def kep2cart(coe, mu) : 

    # TA = true anomaly
    # RA = longitude du noeud ascendant (grand omega)
	a, e, i, w, RA, TA = coe

	h = np.sqrt(mu * a * (1 - e**2))

	rp = (h**2/mu)*(1/(1+e*np.cos(TA)))*np.array([np.cos(TA), np.sin(TA), 0])
	vp = mu/h*np.array([-np.sin(TA), e+np.cos(TA), 0])


	R1 = np.array([ [np.cos(w), -np.sin(w), 0.],
			        [np.sin(w),  np.cos(w), 0.],
			        [             0.,               0., 1.] ])

	R2 = np.array([ [1.,               0.,               0.],
		            [0.,      np.cos(i),     -np.sin(i)],
		            [0.,      np.sin(i),      np.cos(i)] ])

	R3 = np.array([ [np.cos(RA), -np.sin(RA),     0.],
		            [np.sin(RA),  np.cos(RA),     0.],
		            [             0.,               0.,     1.] ])

	M = R3.dot(R2.dot(R1))

	r = M.dot(rp)
	v = M.dot(vp)
	
	return [-r, -v]

def angular_momentum(r_v):
    r = r_v[0]
    v = r_v[1]
    return np.cross(r,v)

In [43]:
## UTILISATION DES FORMULES DU BE OPTIMISATION SPATIALE 


#hyp : orbites circulaires car très faibles e, donc vitesse instantanée constante

# calcul du module de la vitesse sur l'orbite
def calcul_V(a):
    # param gravitationnel standard terreste mu = G*mTerre
    mu = 398600.4418
    V = np.sqrt(mu/a)
    return V

# calcul du module de la vitesse à partir des composantes du vecteur vitesse
def calcul_Vnorm(Vr, Vt, Vn):
    return np.sqrt(Vr**2 + Vt**2 + Vn**2)


def alpha(w, M):
    return w + M

def e_x(e, w):
    return e*np.cos(w)

def e_y(e,w):
    return e*np.sin(w)

def derive_omega(a, i, w, M, e, omega, nu, mass):
    alpha = w + M
    mu = 398600.4418
    orbital_param = (a, e, i, w1, omega, nu)
    position_velocity = kep2cart(orbital_param, mu)
    h = angular_momentum(position_velocity)
    n = LA.norm(np.cross(np.array[0,0,1],h))
    position = position_velocity[0]
    Fgravi = ((mu*mass)/LA.norm(position)**3)*position
    W = np.dot(Fgravi,h)/LA.norm(h)
    derive = (np.sin(alpha)*W)/(n*a*np.sin(i))
    return derive



# dérive relative du RAAN (effet gravitationnel J2) entre deux orbites
def derive_relative(derive1,derive2):
    return derive1 - derive2


# modification du demi grand-axe 
def deltaV_a(a1, a2):
    V = calcul_V(a1)
    delta_a = np.abs(a1 - a2)
    delta_Vt = (delta_a*V)/(2*a1)
    return delta_Vt

# modification de l'inclinaison (à faire à l'un des noeuds)
def deltaV_i(i1, i2, V1):
    delta_i = np.abs(i2 - i1)
    delta_Vn = delta_i*V1
    return delta_Vn

# modification de l'argument du périgée (à faire quand l'anomalie vraiei vaut +- delta_w/2 + k*pi)
def deltaV_w(w1, w2, omega, a, e, i, mass):
    mu = 398600.4418
    delta_w = np.abs(w2 - w1)
    nu = delta_w/2
    orbital_param = (a, e, i, w1, omega, nu)
    position_velocity = kep2cart(orbital_param, mu)
    L = angular_momentum(position_velocity)
    deltaV = (1/L)*2*mu*e*np.sin(delta_w/2)
    return LA.norm(deltaV)

In [25]:
# Exemple entre deux orbites : 

# Etapes : 
# - d'abord laisser dériver RAAN
# - puis manoeuvre pour avoir i = i2
# - puis manoeuvre pour avoir w = w2
# - puis manoeuvre pour avoir a = a2 

# orbite 1 (orbite initiale) : 

debris_data.values[0]


0.0017406

In [45]:


a, e, i, omega, w, M, mass = debris_data.values[0][0], debris_data.values[0][1], debris_data.values[0][2], debris_data.values[0][3], debris_data.values[0][4], debris_data.values[0][5], debris_data.values[0][7]


a2, e2, i2, omega2, w2, M2, mass2 = debris_data.values[11][0], debris_data.values[11][1], debris_data.values[11][2], debris_data.values[11][3], debris_data.values[11][4], debris_data.values[11][5], debris_data.values[11][7] 

#Coût pour la manoeuvre sur i : 
V1 = calcul_V(a)
delta_V_i = deltaV_i(i, i2, V1)
print(delta_V_i)
i = i2

#Coût pour la manoeuvre sur w : 

delta_V_w = deltaV_w(w, w2, omega, a, e, i, mass)
print(delta_V_w)
w = w2

#Coût pour la manoeuvre sur a :

delta_V_a = deltaV_a(a, a2)
print(delta_V_a)
a = a2




1.5220260111921418
0.05179145536361104
0.07359365287466696


In [52]:
def deltaV_tot(i,j):
    a, e, i, omega, w, M, mass = debris_data.values[i][0], debris_data.values[i][1], debris_data.values[i][2], debris_data.values[i][3], debris_data.values[i][4], debris_data.values[i][5], debris_data.values[i][7]

    a2, e2, i2, omega2, w2, M2, mass2 = debris_data.values[j][0], debris_data.values[j][1], debris_data.values[j][2], debris_data.values[j][3], debris_data.values[j][4], debris_data.values[j][5], debris_data.values[j][7] 

    #Coût pour la manoeuvre sur i : 
    V1 = calcul_V(a)
    delta_V_i = deltaV_i(i, i2, V1)
    #print(delta_V_i)
    i = i2

    #Coût pour la manoeuvre sur w : 

    delta_V_w = deltaV_w(w, w2, omega, a, e, i, mass)
    #print(delta_V_w)
    w = w2

    #Coût pour la manoeuvre sur a :

    delta_V_a = deltaV_a(a, a2)
    #print(delta_V_a)
    a = a2

    delta_V_tot = delta_V_i + delta_V_w + delta_V_a
    return delta_V_tot

dict = {}
for i in range(len(debris_data)):
    for j in range (i+1, len(debris_data)):
        dict[f"Total delta V between debris number {i} and {j} is "] = deltaV_tot(i,j)

In [53]:
print(dict)

{'Total delta V between debris number 0 and 1 is ': 0.10328134155637878, 'Total delta V between debris number 0 and 2 is ': 0.13159968342320966, 'Total delta V between debris number 0 and 3 is ': 0.1134141287825251, 'Total delta V between debris number 0 and 4 is ': 0.14714111778849306, 'Total delta V between debris number 0 and 5 is ': 0.18967845727238605, 'Total delta V between debris number 0 and 6 is ': 0.04506084569704085, 'Total delta V between debris number 0 and 7 is ': 0.3910809525143832, 'Total delta V between debris number 0 and 8 is ': 0.15664268505777823, 'Total delta V between debris number 0 and 9 is ': 0.19846933861049018, 'Total delta V between debris number 0 and 10 is ': 1.6843332158506596, 'Total delta V between debris number 0 and 11 is ': 1.6474111194304197, 'Total delta V between debris number 0 and 12 is ': 0.17501266928622353, 'Total delta V between debris number 0 and 13 is ': 0.1093635929302983, 'Total delta V between debris number 0 and 14 is ': 0.1936592804